In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

from scikeras.wrappers import KerasClassifier

In [2]:
data = pd.read_csv('Churn_Modelling.csv')

In [3]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

onehotencoder = OneHotEncoder()
geo_encoder = onehotencoder.fit_transform(data[['Geography']])

geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehotencoder.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)

X = data.drop('Exited',axis=1)
y = data['Exited']

In [4]:
#split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#scale the features

scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [6]:
# save the encoders
import pickle
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehotencoder_geo.pkl','wb') as file:
    pickle.dump(onehotencoder,file)

In [7]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [30]:
def create_model(neurons=32,layers=1):

    
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers - 1):
        model.add(Dense(neurons,activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [31]:

model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [32]:
param_grid={
'neurons':[16,32,64,128],
'layers':[1,2],
'epochs':[50,100]

}

In [33]:
grid = GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)


In [34]:
result = grid.fit(X_train,y_train)
print(result.best_score_)
print(result.best_params_)


c:\Users\Neha Singhal\OneDrive\Desktop\Python Projects\Ann\venv\Lib\site-packages\sklearn\utils\_tags.py:354: FutureWarning: The KerasClassifier or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


AttributeError: 'super' object has no attribute '__sklearn_tags__'